# Introduction to Pandas

`pandas` is an awesome library for csv file manipulations, and so much more!

## First, setup

In [1]:
import pandas as pd
import numpy as np

## Series

The basic `pandas` data structure is `Series`. It represents a one-dimensional collection of values (similar to python native `list` and `numpy` `array`)

In [2]:
s = pd.Series([1, 2, 3])

In [3]:
print(s)

0    1
1    2
2    3
dtype: int64


## Data Frame

`DataFrame` would be the main way to interact with csv files (**tables**). Let's see an example

### First, read from csv file (or any seperated files)

In [6]:
df = pd.read_csv("data/4123_IST_sampling.csv")
df.head()

,participant_id,session,trial_no,global_time_onset_trial,ready_screen_on,ready_screen_duration,category_of_pics,probability,reward_type,majority_cat,...,sample_no,picture_path,pic_name,choose_sample_time,global_choose_sample,picture_on_global,picture_on_trial,picture_off_global,picture_off_trial,
0,4123,1,1,6.175991,6.176686,3.057400,ioc,0.6,5,indoor,...,1,C:/Users/EYETRACKER/Documents/info_sample_tas...,outdoor_82.jpg,1.058262,10.303376,10.332037,4.155356,12.341411,6.164731,
1,4123,1,1,6.175991,6.176686,3.057400,ioc,0.6,5,indoor,...,2,C:/Users/EYETRACKER/Documents/info_sample_tas...,outdoor_133.jpg,0.977646,13.567511,13.593086,7.416406,15.608749,9.432069,
2,4123,1,1,6.175991,6.176686,3.057400,ioc,0.6,5,indoor,...,3,C:/Users/EYETRACKER/Documents/info_sample_tas...,indoor_122.jpg,0.909801,16.767339,16.787396,10.610716,18.796897,12.620216,
3,4123,1,1,6.175991,6.176686,3.057400,ioc,0.6,5,indoor,...,4,C:/Users/EYETRACKER/Documents/info_sample_tas...,outdoor_114.jpg,0.777908,19.823399,19.844268,13.667587,21.853322,15.676641,
4,4123,1,2,26.578054,26.578616,3.051034,lnc,0.6,1,nonliving,...,1,C:/Users/EYETRACKER/Documents/info_sample_tas...,living_152.jpg,0.841852,30.479303,30.497736,3.919125,32.506476,5.927865,
